<a href="https://colab.research.google.com/github/ebwitherspoon/PITT_SEISMIC_AP/blob/master/P2_RQ1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Inclusion criteria: https://docs.google.com/spreadsheets/d/1SzU4PcIEUsAGnKKyAcugHO2O2aZW29sf9a_cC-FAElk/edit#gid=1702351745

In [0]:
# Load libraries
install.packages("pacman")
pacman::p_load("tidyverse", "dplyr", "data.table", "psych") # data wrangling/descriptive stats

In [0]:
# Need sjstats for reporting robust SE
# In R Studio you can just do install.packages("sjstats")
# library(sjstats)

In [0]:
# Load renamed dataset
apfull_SkipEl <- read.csv("/Users/thicn/Documents/AP Skip/apfull_SkipEl.csv")

In [0]:
names(apfull_SkipEl)
head(apfull_SkipEl)

[1] "X.1"                  "X"                    "st_id"               
 [4] "studentid"            "apskipper_GChem1"     "apskipper_Phys1"     
 [7] "aptaker_Chem"         "apyear_Chem"          "apscore_Chem"        
[10] "aptaker_Mec"          "aptaker_Elec"         "apyear_Mec"          
[13] "apyear_Elec"          "apscore_Mec"          "apscore_Elec"        
[16] "coursecode"           "crs_name"             "crs_catalog"         
[19] "class_number"         "crs_sbj"              "level"               
[22] "newfresh"             "numgrade"             "gender"              
[25] "ethniccode"           "ethniccode_cat"       "ethniccode_cat2"     
[28] "firstgen"             "lowincomeflag"        "ellflag"             
[31] "international"        "hsgpa"                "hsgpa_z"             
[34] "bstsat1mathsr"        "bstsat1readsr"        "actenglsr"           
[37] "actmathsr"            "mathsr"               "englsr"              
[40] "crs_term"             "admitdate"            "enrl_from_cohort"    
[43] "yearsenrolled"        "cohort"               "stemmajor"           
[46] "chemNotSkipthoughEl"  "aprealskipper_GChem1" "physNotSkipthoughEl" 
[49] "aprealskipper_Phys1"

X.1,X,st_id,studentid,apskipper_GChem1,apskipper_Phys1,aptaker_Chem,apyear_Chem,apscore_Chem,aptaker_Mec,...,crs_term,admitdate,enrl_from_cohort,yearsenrolled,cohort,stemmajor,chemNotSkipthoughEl,aprealskipper_GChem1,physNotSkipthoughEl,aprealskipper_Phys1
1,1,25,10006293,0,0,0,0,0,0,...,2014,F14,1,0.25,2014,2,0,0,0,0
2,2,25,10006293,0,0,0,0,0,0,...,2014,F14,1,0.25,2014,2,0,0,0,0
3,3,25,10006293,0,0,0,0,0,0,...,2014,F14,1,0.25,2014,2,0,0,0,0
4,4,25,10006293,0,0,0,0,0,0,...,2015,F14,4,1.00,2014,2,0,0,0,0
5,5,25,10006293,0,0,0,0,0,0,...,2015,F14,5,1.25,2014,2,0,0,0,0
6,6,25,10006293,0,0,0,0,0,0,...,2015,F14,5,1.25,2014,2,0,0,0,0


In [0]:
# Notes: variable names all follow convention, except for hsgpa_z = scale(hsgpa)

In [0]:
table(apfull_SkipEl$apyear_Chem)


     0   2007   2008   2009   2010   2011   2012   2013   2014   2015   2016 
324238     11     21    206   2246   7992  13651  13992  15547  13597   5457 
  2017 
   946 

In [0]:
## Start with CHEM
# Create a dataset with unique student ID for those who took the second Chem course
# Here I retain those who didn't take the AP or only took it from 2014 onward.
chemap_2 <- subset(apfull_SkipEl, class_number=="0001B" & crs_name=="GENERAL CHEMISTRY" &
                  apyear_Chem%in%c("0", "2014", "2015", "2016", "2017"))
dim(chemap_2)
table(chemap_2$apyear_Chem)

chemap_unique <- chemap_2[!duplicated(chemap_2$st_id),]
dim(chemap_unique)

table(chemap_unique$aprealskipper_GChem1, chemap_unique$apscore_Chem)

[1] 8274   49


   0 2014 2015 2016 
7111  552  465  146 

[1] 7631   49

   
       0    1    2    3    4    5
  0 6501  194  361  286   20    8
  1    0    0    0    0  111   22

In [0]:
# Recode variables
chemap_unique$aptaker_Chem <- as.factor(chemap_unique$aptaker_Chem)
table(chemap_unique$aptaker_Chem)


   0    1 
6501 1130 

In [0]:
chemap_unique <- subset(chemap_unique, crs_term!="2013")
table(chemap_unique$crs_term)
class(chemap_unique$crs_term)
chemap_unique$crs_term <- as.factor(chemap_unique$crs_term)


2014 2015 2016 2017 
1347 1395 1593 1843 

[1] "integer"

In [0]:
# RQ1. 1a: Took AP, full sample
tookchem <- glm(aptaker_Chem~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
                  hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chemap_unique, binomial(link="logit"))
summary(tookchem)
#robustse(tookchem, coef="odd.ratio") #robustse is a command from sjstats package


Call:
glm(formula = aptaker_Chem ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = chemap_unique)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.43021  -0.71338  -0.43025  -0.00011   2.85923  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -18.86425  175.27318  -0.108  0.91429    
factor(firstgen)1          0.01288    0.08153   0.158  0.87446    
factor(lowincomeflag)2     0.21538    0.08080   2.666  0.00768 ** 
factor(gender)1           -0.06286    0.07646  -0.822  0.41097    
factor(ethniccode_cat2)1   0.13952    0.10261   1.360  0.17394    
hsgpa_z                    0.27919    0.04934   5.659 1.52e-08 ***
scale(mathsr)              0.55129    0.04509  12.227  < 2e-16 ***
scale(englsr)             -0.09655    0.04325  -2.232  0.02559 *  
fact

In [0]:
# 1b. AP score for those who took AP
# Recode numeric variable to numeric
chemap_unique$apscore_Chem <- as.numeric(as.character(chemap_unique$apscore_Chem)) 
# Subset for those who took AP
chem_tookap <- subset(chemap_unique, aptaker_Chem!=0)
dim(chem_tookap)
table(chem_tookap$apscore_Chem)

# Logit model
chemsr <- lm(apscore_Chem~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chem_tookap)
summary(chemsr)
#std_beta(chemsr) # from sjstats package

[1] 1130   49


  1   2   3   4   5 
194 361 286 131  30 


Call:
lm(formula = apscore_Chem ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), data = chem_tookap)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.07074 -0.59024 -0.03947  0.54309  3.00445 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.51606    0.10827  23.238  < 2e-16 ***
factor(firstgen)1        -0.13947    0.06309  -2.211 0.027279 *  
factor(lowincomeflag)2    0.03259    0.06065   0.537 0.591097    
factor(gender)1          -0.14339    0.05725  -2.505 0.012418 *  
factor(ethniccode_cat2)1  0.05695    0.07715   0.738 0.460595    
hsgpa_z                   0.13675    0.03959   3.454 0.000576 ***
scale(mathsr)             0.41767    0.03296  12.673  < 2e-16 ***
scale(englsr)             0.19243    0.03282   5.864  6.2e-09 ***
factor(crs_term)2016      0.08334    0.08815   0.945 0.344675    
factor(crs

In [0]:
# 1c.i. Eligible out of those who took AP
table(chem_tookap$apskipper_GChem1)
skipchem <- glm(apskipper_GChem1~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=chem_tookap, 
                binomial(link="logit"))
summary(skipchem)
#robustse(skipchem, coef="odd.ratio")


  0   1 
969 161 


Call:
glm(formula = apskipper_GChem1 ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = chem_tookap)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8048  -0.5347  -0.3029  -0.1629   2.9019  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -2.3161     0.3924  -5.903 3.57e-09 ***
factor(firstgen)1         -0.5804     0.2424  -2.394  0.01667 *  
factor(lowincomeflag)2     0.2736     0.2265   1.208  0.22704    
factor(gender)1           -0.6412     0.1990  -3.222  0.00127 ** 
factor(ethniccode_cat2)1   0.4911     0.2766   1.775  0.07589 .  
hsgpa_z                    0.4542     0.1577   2.880  0.00397 ** 
scale(mathsr)              0.8932     0.1339   6.669 2.58e-11 ***
scale(englsr)              0.5301     0.1159   4.575 4.75e-06 ***
factor(crs_term)2016 

In [0]:
# 1c.ii. Eligible out of full sample
skipchem2 <- glm(apskipper_GChem1~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=chemap_unique, 
                binomial(link="logit"))
summary(skipchem2)
#robustse(skipchem2, coef="odd.ratio")


Call:
glm(formula = apskipper_GChem1 ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = chemap_unique)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2576  -0.2030  -0.0964  -0.0221   3.5199  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -21.43391  437.76156  -0.049   0.9609    
factor(firstgen)1         -0.52043    0.21878  -2.379   0.0174 *  
factor(lowincomeflag)2     0.35525    0.20316   1.749   0.0804 .  
factor(gender)1           -0.48361    0.17702  -2.732   0.0063 ** 
factor(ethniccode_cat2)1   0.56468    0.24764   2.280   0.0226 *  
hsgpa_z                    0.53566    0.13507   3.966 7.32e-05 ***
scale(mathsr)              1.13362    0.11431   9.917  < 2e-16 ***
scale(englsr)              0.41324    0.09999   4.133 3.58e-05 ***
factor(crs

In [0]:
# 2a. Took AP AND skip eligble and actually skip
table(chem_tookap$aprealskipper_GChem1)
# Make skip eligible data subset
skipel <- subset(chem_tookap, apskipper_GChem1==1)
dim(skipel)

skipreal <- glm(aprealskipper_GChem1~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=skipel, 
                binomial(link="logit"))
summary(skipreal)
#robustse(skipreal, coef="odd.ratio")


  0   1 
997 133 

[1] 161  49


Call:
glm(formula = aprealskipper_GChem1 ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = skipel)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.04741   0.00002   0.00004   0.00017   1.63527  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)   
(Intercept)                21.21590 3541.01745   0.006  0.99522   
factor(firstgen)1          -1.07641    0.66003  -1.631  0.10293   
factor(lowincomeflag)2      0.04497    0.69560   0.065  0.94845   
factor(gender)1             0.20344    0.60228   0.338  0.73552   
factor(ethniccode_cat2)1    0.49697    0.86541   0.574  0.56579   
hsgpa_z                    -0.73650    0.56828  -1.296  0.19497   
scale(mathsr)               0.87313    0.33765   2.586  0.00971 **
scale(englsr)              -0.47297    0.35175  -1.345  0.17875   
fac

In [0]:
# 2b. Took AP, include AP score
chemsr5 <- lm(numgrade~aprealskipper_GChem1 + apscore_Chem +
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
              hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chem_tookap)
summary(chemsr5)

#std_beta(chemsr5)


Call:
lm(formula = numgrade ~ aprealskipper_GChem1 + apscore_Chem + 
    factor(firstgen) + factor(lowincomeflag) + factor(gender) + 
    factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + scale(englsr) + 
    factor(crs_term), data = chem_tookap)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6063 -0.4137  0.1182  0.5201  1.6060 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.3034529  0.1250849  18.415  < 2e-16 ***
aprealskipper_GChem1      0.0007455  0.0945423   0.008 0.993710    
apscore_Chem              0.2230602  0.0348782   6.395 2.49e-10 ***
factor(firstgen)1         0.1126677  0.0566239   1.990 0.046899 *  
factor(lowincomeflag)2   -0.1084974  0.0542264  -2.001 0.045690 *  
factor(gender)1          -0.0628662  0.0514196  -1.223 0.221775    
factor(ethniccode_cat2)1 -0.0927188  0.0690454  -1.343 0.179630    
hsgpa_z                   0.2483772  0.0355816   6.981 5.46e-12 ***
scale(mathsr)             0.1

In [0]:
# 2c. Full sample, include AP score
chemsr6 <- lm(numgrade~aprealskipper_GChem1 + apscore_Chem +
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
              hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chemap_unique)
summary(chemsr6)

#std_beta(chemsr6)


Call:
lm(formula = numgrade ~ aprealskipper_GChem1 + apscore_Chem + 
    factor(firstgen) + factor(lowincomeflag) + factor(gender) + 
    factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + scale(englsr) + 
    factor(crs_term), data = chemap_unique)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6050 -0.4678  0.0844  0.6075  2.5635 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.51071    0.04121  60.919  < 2e-16 ***
aprealskipper_GChem1      0.05636    0.09585   0.588  0.55654    
apscore_Chem              0.09686    0.01466   6.606 4.29e-11 ***
factor(firstgen)1        -0.03303    0.02563  -1.288  0.19764    
factor(lowincomeflag)2    0.01624    0.02544   0.638  0.52327    
factor(gender)1          -0.09626    0.02464  -3.906 9.49e-05 ***
factor(ethniccode_cat2)1 -0.13446    0.03241  -4.149 3.38e-05 ***
hsgpa_z                   0.25231    0.01458  17.306  < 2e-16 ***
scale(mathsr)             0.27713    0.01431 

In [0]:
# 2d. 2nd course grade; took AP sample for those who actually skip
chemsr <- lm(numgrade~aprealskipper_GChem1 + 
             factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
              hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
             data=chem_tookap)
summary(chemsr)

#std_beta(chemsr)


Call:
lm(formula = numgrade ~ aprealskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = chem_tookap)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6137 -0.4322  0.1119  0.5380  1.7016 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.762432   0.094409  29.260  < 2e-16 ***
aprealskipper_GChem1      0.321516   0.078785   4.081 4.81e-05 ***
factor(firstgen)1         0.078870   0.054288   1.453   0.1466    
factor(lowincomeflag)2   -0.089312   0.052396  -1.705   0.0886 .  
factor(gender)1          -0.060543   0.049857  -1.214   0.2249    
factor(ethniccode_cat2)1 -0.102681   0.066996  -1.533   0.1257    
hsgpa_z                   0.280741   0.033943   8.271 3.84e-16 ***
scale(mathsr)             0.239618   0.029459   8.134 1.12e-15 ***
scale(englsr)             0.008505   0.028913   0.294

In [0]:
#2e. 2nd course grade; everyone, actually skipped
#chemap_unique$numgrade <- as.numeric(as.character(chemap_unique$numgrade))
chemsr2 <- lm(numgrade ~ aprealskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chemap_unique)
summary(chemsr2)

#std_beta(chemsr2)


Call:
lm(formula = numgrade ~ aprealskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = chemap_unique)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6351 -0.4802  0.0945  0.6160  2.6109 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.50338    0.04104  61.000  < 2e-16 ***
aprealskipper_GChem1      0.38602    0.08102   4.764 1.94e-06 ***
factor(firstgen)1        -0.03257    0.02542  -1.281 0.200104    
factor(lowincomeflag)2    0.02256    0.02522   0.895 0.371004    
factor(gender)1          -0.09209    0.02445  -3.766 0.000168 ***
factor(ethniccode_cat2)1 -0.13344    0.03216  -4.149 3.38e-05 ***
hsgpa_z                   0.26120    0.01444  18.083  < 2e-16 ***
scale(mathsr)             0.29605    0.01402  21.109  < 2e-16 ***
scale(englsr)             0.02047    0.01388   1.475 0.14027

In [0]:
#2f. Course grade, Skip eligible for those who took AP
chemsr3 <- lm(numgrade~apskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chem_tookap)
summary(chemsr3)

#std_beta(chemsr3)


Call:
lm(formula = numgrade ~ apskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = chem_tookap)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5774 -0.4338  0.1104  0.5419  1.6928 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.771220   0.094298  29.388  < 2e-16 ***
apskipper_GChem1          0.280368   0.072937   3.844 0.000128 ***
factor(firstgen)1         0.075936   0.054294   1.399 0.162217    
factor(lowincomeflag)2   -0.089152   0.052445  -1.700 0.089429 .  
factor(gender)1          -0.061241   0.049908  -1.227 0.220060    
factor(ethniccode_cat2)1 -0.101853   0.067054  -1.519 0.129057    
hsgpa_z                   0.278394   0.034024   8.182 7.70e-16 ***
scale(mathsr)             0.241714   0.029447   8.208 6.27e-16 ***
scale(englsr)             0.005873   0.029050   0.202 0.8

In [0]:
#2g. Skip eligible, full sample
chemsr4<- lm(numgrade~apskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chemap_unique)
summary(chemsr4)

#std_beta(chemsr4)


Call:
lm(formula = numgrade ~ apskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = chemap_unique)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6029 -0.4791  0.0964  0.6140  2.6086 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.50382    0.04104  61.012  < 2e-16 ***
apskipper_GChem1          0.35378    0.07384   4.791 1.70e-06 ***
factor(firstgen)1        -0.03311    0.02541  -1.303 0.192639    
factor(lowincomeflag)2    0.02228    0.02522   0.883 0.377069    
factor(gender)1          -0.09233    0.02445  -3.776 0.000161 ***
factor(ethniccode_cat2)1 -0.13324    0.03216  -4.143 3.47e-05 ***
hsgpa_z                   0.26046    0.01445  18.026  < 2e-16 ***
scale(mathsr)             0.29588    0.01403  21.093  < 2e-16 ***
scale(englsr)             0.01986    0.01389   1.430 0.152831   

In [0]:
# Bring back the var names so I don't have to scroll up ...
names(apfull_SkipEl)

[1] "X.1"                  "X"                    "st_id"               
 [4] "studentid"            "apskipper_GChem1"     "apskipper_Phys1"     
 [7] "aptaker_Chem"         "apyear_Chem"          "apscore_Chem"        
[10] "aptaker_Mec"          "aptaker_Elec"         "apyear_Mec"          
[13] "apyear_Elec"          "apscore_Mec"          "apscore_Elec"        
[16] "coursecode"           "crs_name"             "crs_catalog"         
[19] "class_number"         "crs_sbj"              "level"               
[22] "newfresh"             "numgrade"             "gender"              
[25] "ethniccode"           "ethniccode_cat"       "ethniccode_cat2"     
[28] "firstgen"             "lowincomeflag"        "ellflag"             
[31] "international"        "hsgpa"                "hsgpa_z"             
[34] "bstsat1mathsr"        "bstsat1readsr"        "actenglsr"           
[37] "actmathsr"            "mathsr"               "englsr"              
[40] "crs_term"             "admitdate"            "enrl_from_cohort"    
[43] "yearsenrolled"        "cohort"               "stemmajor"           
[46] "chemNotSkipthoughEl"  "aprealskipper_GChem1" "physNotSkipthoughEl" 
[49] "aprealskipper_Phys1"

In [0]:
## PHYSICS
#table(apfull_SkipEl$apyear_Mec) # NA not coded
apfull_SkipEl$apyear_Mec <- ifelse(is.na(apfull_SkipEl$apyear_Mec), 0, apfull_SkipEl$apyear_Mec)
apfull_SkipEl$apyear_Elec <- ifelse(is.na(apfull_SkipEl$apyear_Elec), 0, apfull_SkipEl$apyear_Elec)

table(apfull_SkipEl$apyear_Mec)
table(apfull_SkipEl$apyear_Elec)


     0   2009   2010   2011   2012   2013   2014   2015   2016   2017 
385522     28    233   1120   2053   1973   2255   2947   1537    236 


     0   2009   2010   2011   2012   2013   2014   2015   2016   2017 
394116     18    141    382    674    469    815    900    344     45 

In [0]:
## PHYSICS
dim(apfull_SkipEl)
physicsap <- subset(apfull_SkipEl, class_number=="0007D" & crs_name=="CLASSICAL PHYSICS" & 
                    apyear_Mec%in%c("0", "2015", "2016", "2017") | 
                                    class_number=="0007D" & crs_name=="CLASSICAL PHYSICS" & 
                                    apyear_Elec%in%c("0", "2015", "2016", "2017"))
physicsap <- subset(physicsap, crs_term%in%c("2015", "2016", "2017"))
dim(physicsap)

physicsap_unique <- physicsap[!duplicated(physicsap$st_id),]
dim(physicsap_unique)

table(physicsap_unique$aprealskipper_Phys1, physicsap_unique$apscore_Mec)
table(physicsap_unique$aprealskipper_Phys1, physicsap_unique$apscore_Elec)

[1] 397904     49

[1] 1906   49

[1] 1867   49

   
     1  2  3  4  5
  0 30 49 44 13  7
  1  0  0  0 22 15

   
     1  2  3  4  5
  0  8 14 18  5 12
  1  1  0  1  0  1

In [0]:
# More data processing
# Create a dummy var for taking physics (either Mec or Elec)
physicsap_unique$aptaker_Phys <- ifelse(physicsap_unique$aptaker_Elec==1|
                                    physicsap_unique$aptaker_Mec==1, 1, 0)

# Create a score for physics (as the best score from either Mec or Elec)
# Replace NA with 0
physicsap_unique$apscore_Elec[is.na(physicsap_unique$apscore_Elec)] <- 0
physicsap_unique$apscore_Mec[is.na(physicsap_unique$apscore_Mec)] <- 0
# Choose best score from either elec or mec: first replace with one, i.e., Mec score, and compare with Elec score; if higher then retain
physicsap_unique$apscore_Phys <- physicsap_unique$apscore_Mec

physicsap_unique$apscore_Phys <- ifelse(physicsap_unique$apscore_Elec > physicsap_unique$apscore_Phys, 
                                       physicsap_unique$apscore_Elec, physicsap_unique$apscore_Phys)
table(physicsap_unique$apscore_Phys, physicsap_unique$apscore_Mec) # double checking
table(physicsap_unique$apscore_Phys, physicsap_unique$aprealskipper_Phys1) # real skippers in those that score 4/5 on either test

   
       0    1    2    3    4    5
  0 1661    0    0    0    0    0
  1    4   25    0    0    0    0
  2    7    2   44    0    0    0
  3    8    2    3   40    0    0
  4    3    0    0    2   31    0
  5    4    1    2    2    4   22

   
       0    1
  0 1661    0
  1   29    0
  2   53    0
  3   53    0
  4   15   21
  5   19   16

In [0]:
#1a. Took AP, full sample
tookphysap <- glm(aptaker_Phys~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
                  hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                  data=physicsap_unique, binomial(link="logit"))
summary(tookphysap)
#robustse(tookphysap, coef="odd.ratio")


Call:
glm(formula = aptaker_Phys ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = physicsap_unique)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2183  -0.6694  -0.5148  -0.3395   2.4949  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -1.60033    0.17902  -8.939  < 2e-16 ***
factor(firstgen)1        -0.47841    0.15725  -3.042  0.00235 ** 
factor(lowincomeflag)2    0.41153    0.15538   2.648  0.00809 ** 
factor(gender)1          -0.43611    0.16156  -2.699  0.00695 ** 
factor(ethniccode_cat2)1  0.04109    0.15799   0.260  0.79480    
hsgpa_z                  -0.09636    0.07630  -1.263  0.20660    
scale(mathsr)             0.53468    0.08047   6.645 3.04e-11 ***
scale(englsr)             0.16491    0.07722   2.135  0.03273 *  
factor(crs_term)2016

In [0]:
# 1b: AP Score for those who took AP
phys_tookap <- subset(physicsap_unique, apscore_Phys!=0)
dim(phys_tookap) 
table(phys_tookap$apscore_Phys)

physsr <- lm(apscore_Phys~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=phys_tookap)
summary(physsr)
#std_beta(physsr)

[1] 206  51


 1  2  3  4  5 
29 53 53 36 35 


Call:
lm(formula = apscore_Phys ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), data = phys_tookap)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.31842 -0.97640 -0.04894  1.02794  2.33646 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.77073    0.26670  10.389   <2e-16 ***
factor(firstgen)1        -0.08575    0.26178  -0.328    0.744    
factor(lowincomeflag)2   -0.35349    0.25113  -1.408    0.161    
factor(gender)1          -0.05413    0.24644  -0.220    0.826    
factor(ethniccode_cat2)1  0.30919    0.24535   1.260    0.209    
hsgpa_z                  -0.03570    0.12510  -0.285    0.776    
scale(mathsr)             0.03025    0.10908   0.277    0.782    
scale(englsr)            -0.13718    0.12072  -1.136    0.257    
factor(crs_term)2016      0.24110    0.23730   1.016    0.311    
factor(crs

In [0]:
# 1c.i Eligible out of those who took AP
skipphys <- glm(apskipper_Phys1~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=phys_tookap, 
                binomial(link="logit"))
summary(skipphys)
#robustse(skipphys, coef="odd.ratio")


Call:
glm(formula = apskipper_Phys1 ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = phys_tookap)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.0998  -0.8184  -0.7024   1.3333   2.0724  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)   
(Intercept)              -1.25663    0.47923  -2.622  0.00874 **
factor(firstgen)1        -0.12437    0.46936  -0.265  0.79102   
factor(lowincomeflag)2   -0.49682    0.45194  -1.099  0.27164   
factor(gender)1          -0.64653    0.47057  -1.374  0.16946   
factor(ethniccode_cat2)1  0.34921    0.43096   0.810  0.41776   
hsgpa_z                   0.21514    0.22538   0.955  0.33981   
scale(mathsr)            -0.06921    0.19018  -0.364  0.71594   
scale(englsr)            -0.10250    0.21086  -0.486  0.62689   
factor(crs_term)2016      0.223

In [0]:
# 1c.ii Eligible out of full sample
skipphys2 <- glm(apskipper_Phys1~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=physicsap_unique, 
                binomial(link="logit"))
summary(skipphys2)
#robustse(skipphys2, coef="odd.ratio")


Call:
glm(formula = apskipper_Phys1 ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = physicsap_unique)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.4469  -0.2775  -0.2193  -0.1735   3.1156  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -3.72613    0.42336  -8.801   <2e-16 ***
factor(firstgen)1        -0.43926    0.36084  -1.217   0.2235    
factor(lowincomeflag)2   -0.02844    0.35498  -0.080   0.9361    
factor(gender)1          -0.78314    0.41665  -1.880   0.0602 .  
factor(ethniccode_cat2)1  0.36180    0.36941   0.979   0.3274    
hsgpa_z                   0.05972    0.17634   0.339   0.7349    
scale(mathsr)             0.34920    0.17919   1.949   0.0513 .  
scale(englsr)             0.10489    0.17331   0.605   0.5450    
factor(crs_term)2

In [0]:
# 2a. Took AP and skip eligble and actually skip
table(phys_tookap$aprealskipper_Phys1)
skipel <- subset(phys_tookap, apskipper_Phys1==1)
dim(skipel)
table(skipel$aprealskipper_Phys1)

skipreal <- glm(aprealskipper_Phys1~factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=skipel, 
                binomial(link="logit"))
summary(skipreal) # very small dataset in UCI case | not reliable coefficients.
#robustse(skipreal, coef="odd.ratio")


  0   1 
169  37 

[1] 57 51


 0  1 
20 37 


Call:
glm(formula = aprealskipper_Phys1 ~ factor(firstgen) + factor(lowincomeflag) + 
    factor(gender) + factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + 
    scale(englsr) + factor(crs_term), family = binomial(link = "logit"), 
    data = skipel)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9101  -1.2279   0.7868   0.8969   1.3047  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)
(Intercept)               1.406276   0.991534   1.418    0.156
factor(firstgen)1        -0.382455   0.959847  -0.398    0.690
factor(lowincomeflag)2   -0.304647   0.933738  -0.326    0.744
factor(gender)1          -0.530555   0.940600  -0.564    0.573
factor(ethniccode_cat2)1 -0.244630   0.846912  -0.289    0.773
hsgpa_z                   0.001352   0.446123   0.003    0.998
scale(mathsr)             0.111766   0.367035   0.305    0.761
scale(englsr)            -0.443238   0.415905  -1.066    0.287
factor(crs_term)2016     -0.220520   0.884561  -0.

In [0]:
# 2b. Took AP, include AP score
physsr5 <- lm(numgrade~aprealskipper_Phys1 + apscore_Phys + 
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
              data=phys_tookap)
summary(physsr5)

#std_beta(physsr5)



Call:
lm(formula = numgrade ~ aprealskipper_Phys1 + apscore_Phys + 
    factor(firstgen) + factor(lowincomeflag) + factor(gender) + 
    factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + scale(englsr) + 
    factor(crs_term), data = phys_tookap)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.49243 -0.44263  0.03635  0.52558  1.55960 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               3.22456    0.19362  16.654  < 2e-16 ***
aprealskipper_Phys1      -0.12087    0.16635  -0.727   0.4684    
apscore_Phys             -0.07923    0.04877  -1.625   0.1059    
factor(firstgen)1        -0.33075    0.14811  -2.233   0.0267 *  
factor(lowincomeflag)2    0.16505    0.14279   1.156   0.2492    
factor(gender)1          -0.29347    0.14009  -2.095   0.0375 *  
factor(ethniccode_cat2)1 -0.17276    0.13937  -1.240   0.2167    
hsgpa_z                   0.29367    0.07107   4.132 5.42e-05 ***
scale(mathsr)             0.36786    0

In [0]:
# 2c. Full sample, include AP score
physsr6 <- lm(numgrade~aprealskipper_Phys1 + apscore_Phys + 
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=physicsap_unique)
summary(physsr6)
#std_beta(physsr6)


Call:
lm(formula = numgrade ~ aprealskipper_Phys1 + apscore_Phys + 
    factor(firstgen) + factor(lowincomeflag) + factor(gender) + 
    factor(ethniccode_cat2) + hsgpa_z + scale(mathsr) + scale(englsr) + 
    factor(crs_term), data = physicsap_unique)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.1494 -0.5106  0.0667  0.6141  2.4813 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.66814    0.05638  47.322  < 2e-16 ***
aprealskipper_Phys1      -0.28169    0.17561  -1.604 0.108866    
apscore_Phys              0.06174    0.02399   2.573 0.010147 *  
factor(firstgen)1        -0.11256    0.04685  -2.403 0.016379 *  
factor(lowincomeflag)2    0.08155    0.04758   1.714 0.086695 .  
factor(gender)1          -0.16867    0.04588  -3.676 0.000243 ***
factor(ethniccode_cat2)1 -0.11296    0.05174  -2.183 0.029151 *  
hsgpa_z                   0.22380    0.02446   9.150  < 2e-16 ***
scale(mathsr)             0.22082    0.0243

In [0]:
# 2d. 2nd course grade; took AP sample for those who actually skip
physsr <- lm(numgrade~aprealskipper_Phys1 + factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
             data=phys_tookap)
summary(physsr)

#std_beta(physsr)


Call:
lm(formula = numgrade ~ aprealskipper_Phys1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = phys_tookap)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.44608 -0.44240  0.09435  0.54090  1.61489 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               3.03198    0.15376  19.718  < 2e-16 ***
aprealskipper_Phys1      -0.26510    0.14130  -1.876   0.0622 .  
factor(firstgen)1        -0.32846    0.14875  -2.208   0.0285 *  
factor(lowincomeflag)2    0.18255    0.14301   1.277   0.2034    
factor(gender)1          -0.30264    0.14059  -2.153   0.0326 *  
factor(ethniccode_cat2)1 -0.19301    0.13942  -1.384   0.1679    
hsgpa_z                   0.30141    0.07122   4.232 3.62e-05 ***
scale(mathsr)             0.36440    0.06196   5.881 1.85e-08 ***
scale(englsr)            -0.10394    0.06876  -1.512 

In [0]:
#2e. 2nd course grade; everyone, actually skipped
physsr2 <- lm(numgrade~aprealskipper_Phys1 + factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=physicsap_unique)
summary(physsr2)

#std_beta(physsr2)


Call:
lm(formula = numgrade ~ aprealskipper_Phys1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = physicsap_unique)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.1801 -0.5076  0.0732  0.6111  2.4705 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.68068    0.05626  47.649  < 2e-16 ***
aprealskipper_Phys1      -0.02452    0.14462  -0.170 0.865382    
factor(firstgen)1        -0.11580    0.04690  -2.469 0.013640 *  
factor(lowincomeflag)2    0.08711    0.04760   1.830 0.067409 .  
factor(gender)1          -0.17006    0.04594  -3.701 0.000221 ***
factor(ethniccode_cat2)1 -0.10928    0.05180  -2.110 0.035018 *  
hsgpa_z                   0.22130    0.02448   9.041  < 2e-16 ***
scale(mathsr)             0.22723    0.02426   9.366  < 2e-16 ***
scale(englsr)             0.02753    0.02456   1.121 0.262

In [0]:
#2f. Skip eligible for those who took AP
physsrsr3 <- lm(numgrade~apskipper_Phys1 + 
                factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=phys_tookap)
summary(physsrsr3)

#std_beta(physsrsr3)


Call:
lm(formula = numgrade ~ apskipper_Phys1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = phys_tookap)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.46542 -0.44816  0.07446  0.52640  1.60620 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               3.03545    0.15388  19.726  < 2e-16 ***
apskipper_Phys1          -0.23505    0.12195  -1.927   0.0554 .  
factor(firstgen)1        -0.32540    0.14863  -2.189   0.0298 *  
factor(lowincomeflag)2    0.18000    0.14301   1.259   0.2097    
factor(gender)1          -0.30483    0.14060  -2.168   0.0314 *  
factor(ethniccode_cat2)1 -0.18513    0.13952  -1.327   0.1862    
hsgpa_z                   0.30204    0.07119   4.243 3.47e-05 ***
scale(mathsr)             0.36312    0.06195   5.862 2.03e-08 ***
scale(englsr)            -0.09901    0.06857  -1.444   0.

In [0]:
#2g. Skip eligible, full sample
physsr4<- lm(numgrade~apskipper_Phys1 + 
                factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=physicsap_unique)
summary(physsr4)

#std_beta(physsr4)


Call:
lm(formula = numgrade ~ apskipper_Phys1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat2) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
    data = physicsap_unique)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.1783 -0.5144  0.0725  0.6069  2.4708 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               2.67972    0.05626  47.631  < 2e-16 ***
apskipper_Phys1           0.01653    0.11833   0.140 0.888941    
factor(firstgen)1        -0.11540    0.04691  -2.460 0.013974 *  
factor(lowincomeflag)2    0.08715    0.04760   1.831 0.067307 .  
factor(gender)1          -0.16945    0.04595  -3.688 0.000233 ***
factor(ethniccode_cat2)1 -0.10959    0.05181  -2.115 0.034537 *  
hsgpa_z                   0.22124    0.02448   9.039  < 2e-16 ***
scale(mathsr)             0.22693    0.02427   9.351  < 2e-16 ***
scale(englsr)             0.02749    0.02456   1.119 0.263078 